In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
keyword_df = pd.read_pickle('../data/keywords.pk')


In [5]:
keyword_df

,Name,KeywordsTitle,Keywords
RecipeId,,,
38,low-fat berry blue frozen dessert,[dessert],"[frozen desserts, healthy, freezer, dessert, s..."
40,best lemonade,[lemon],"[healthy, summer, lemon, beverages]"
42,cabbage soup,"[cabbage, soup]","[healthy, vegetable, winter, vegan, cabbage, e..."
44,warm chicken a la king,[chicken],"[noodle, pie, potato, rice, chicken, meat, pou..."
45,buttermilk pie with gingersnap crumb crust,[pie],"[pie, healthy, dessert, weeknight, oven]"
...,...,...,...
539182,cinnamony sweet moroccan orange salad,"[salad, moroccan, sweet]","[citrus, healthy, fruit, oranges, sweet, salad..."
539184,belgian chocolate-fudge sauce,"[chocolate, belgian, sauce]","[waffle, sauce, chocolate, sauces, belgian, de..."
539304,pork tenderloin with bacon-onion jam,[pork],"[pork, meat]"


In [3]:
keyword_list_orig = []
for kwl in keyword_df.Keywords:
    keyword_list_orig+=list(kwl)

keyword_list_orig_nr = list(set(keyword_list_orig))

In [19]:
appliances=['microwave', 'blender','air fryer', 'oven', 'stove','toaster', 'mixer','pot',
          'pan', 'kettle', 'rice cooker','cutting board', 'processor', 'peeler',
          'spatula','tongs','colander','baking sheet','grater', 'cup','meat grinder','juicer',
          'bowl', 'freezer', 'whisk', 'electric', 'grill', 'bread machine', 'convection oven', 'dutch oven'
          'deep fryer', 'crepe maker', 'roaster', 'steamer', 'griddle', 'hot plate', 
          'instant pot', 'pressure cooker', 'crock pot', 'flattop grill', 'slow cooker', 
          'sous-vide cooker', 'stove', 'waffle iron', 'baster', 'blow torch', 'can opener', 
          'shredder', 'cheesecloth', 'clay pot', 'corkscrew', 'poacher', 'funnel', 'mill', 
          'sifter', 'garlic press', 'strainer', 'ladle', 'mandoline', 'tenderiser', 'meat thermometer'
          'microplane', 'mortar', 'pestle', 'frother', 'pastry brush', 'masher', 'shears', 'rolling pin',
          'scale', 'sieve', 'spider', 'thermometer', 'twine', 'zester']

In [20]:
len([r for r in keyword_df.index if 'pan' in keyword_df['Name'][r]])

2388

This seems small for a very common word in recipes. It must not be listed as a keyword as frequently as it is mentioned in a recipe.

In [11]:
len([r for r in keyword_df.index if 'oven' in keyword_df['Name'][r]])

886

In [36]:
len([r for r in keyword_df.index if 'dutch oven' in keyword_df['Name'][r]])

13

In [12]:
len([r for r in keyword_df.index if 'crock pot' in keyword_df['Name'][r]])

2096

Most of the appliance keywords indicate basic items like oven and pan. I want to find a way to make sure the appliance list includes every appliance actually in the recipe, not just these basic items.

In [21]:
keyword_df['ApplianceList'] = [set(appliances).intersection(set(kws)) for kws in keyword_df.Keywords]

In [22]:
appliances_df = keyword_df.loc[keyword_df['ApplianceList'].apply(len)>0][['Name','ApplianceList']].copy()

In [24]:
appliances_df

,Name,ApplianceList
RecipeId,,
38,low-fat berry blue frozen dessert,{freezer}
45,buttermilk pie with gingersnap crumb crust,{oven}
49,chicken breasts lombardi,{oven}
54,carrot cake,{oven}
56,buttermilk pie,{oven}
...,...,...
528649,jack-o-lantern cake,{oven}
533244,johnny cakes / journey cakes,{oven}
536843,sheet pan turkey caprese meatballs with rosema...,{oven}


I am not sure why this is choosing just one appliance for each recipe. It is possible that very few appliances are appearing in the keyword file.

In [25]:
keyword_df['ApplianceExt'] = [set(appliances).intersection(set(kws)) for kws in keyword_df.Keywords]

In [26]:
keyword_df['ApplianceExt']

RecipeId
38        {freezer}
40               {}
42               {}
44               {}
45           {oven}
            ...    
539182           {}
539184           {}
539304           {}
540263           {}
540370           {}
Name: ApplianceExt, Length: 114744, dtype: object

In [27]:
keyword_df['ApplianceList']

RecipeId
38        {freezer}
40               {}
42               {}
44               {}
45           {oven}
            ...    
539182           {}
539184           {}
539304           {}
540263           {}
540370           {}
Name: ApplianceList, Length: 114744, dtype: object

In [29]:
unknown_appliances = keyword_df.loc[keyword_df['ApplianceList'].apply(len)==0][['Name']].copy()

In [30]:
appliances_df['ApplianceKeywords'] = [list(set(appliances_df['ApplianceList'][r])) for r in appliances_df.index]

In [31]:
def appliance_vec(rkwl:list)->list:
    vec = [0]*len(appliances)
    for i in range(len(appliances)):
        if appliances[i] in rkwl:
            vec[i]+=1
    return vec


In [32]:
keyword_df.loc[keyword_df['ApplianceExt'].apply(len)>0]

,Name,KeywordsTitle,Keywords,ApplianceList,ApplianceExt
RecipeId,,,,,
38,low-fat berry blue frozen dessert,[dessert],"[frozen desserts, healthy, freezer, dessert, s...",{freezer},{freezer}
45,buttermilk pie with gingersnap crumb crust,[pie],"[pie, healthy, dessert, weeknight, oven]",{oven},{oven}
49,chicken breasts lombardi,"[chicken breast, chicken]","[poultry, chicken breast, chicken, weeknight, ...",{oven},{oven}
54,carrot cake,[cake],"[vegetable, cake, dessert, weeknight, oven]",{oven},{oven}
56,buttermilk pie,[pie],"[pie, bear, dessert, sweet, easy, oven]",{oven},{oven}
...,...,...,...,...,...
528649,jack-o-lantern cake,[cake],"[for large groups, pie, cake, baking, hallowee...",{oven},{oven}
533244,johnny cakes / journey cakes,[cake],"[deep fried, breads, coconut, cake, fried, bak...",{oven},{oven}
536843,sheet pan turkey caprese meatballs with rosema...,"[meat, potato, meatballs]","[potato, meatballs, vegetable, poultry, mashed...",{oven},{oven}


In [33]:
keyword_df['ApplianceKeywords']=keyword_df['ApplianceList']
appliance_df = keyword_df.loc[keyword_df['ApplianceKeywords'].apply(len)>0][['Name','ApplianceKeywords']].copy()

In [34]:
appliance_df

,Name,ApplianceKeywords
RecipeId,,
38,low-fat berry blue frozen dessert,{freezer}
45,buttermilk pie with gingersnap crumb crust,{oven}
49,chicken breasts lombardi,{oven}
54,carrot cake,{oven}
56,buttermilk pie,{oven}
...,...,...
528649,jack-o-lantern cake,{oven}
533244,johnny cakes / journey cakes,{oven}
536843,sheet pan turkey caprese meatballs with rosema...,{oven}


In [35]:
appliance_df.to_pickle('../data/clean_columns/appliance_clean.pk')